<html><h1 style="background: -webkit-gradient(linear,left top,left bottom,from(#fff),to(#efefef)); border-radius: 10px; box-shadow: 0 0 4px;
width: 100%; border: 1px solid grey; padding: 1em; box-sizing: border-box; margin-top: 5px; margin-bottom: 5px; text-align: center; background: brown; color: white; letter-spacing: 2px; font-weight: bold;line-height: 48px;">Integer Triangles Trait Cards<br>Computed with Julia<br><a href="http://luschny.de/julia/triangles/TutorialTrianglesPart1.html">(See also the Tutorial)</a></h1>

In [1]:
# Pkg.add(PackageSpec(url="https://github.com/OpenLibMathSeq/IntegerTriangles.jl"))
# Pkg.add("IntegerTriangles.jl"))

In [ ]:
using IntegerTriangles

Definition: A *integer triangle* is an array of arrays whose members are integers. 
It has the type **ℤTri**.

An integer is a multiple precision integer which is created by the constructor **ℤInt**.

The row of an integer triangle is an integer sequence, **ℤSeq**.

Examples for the creation of an integer triangle:

In [3]:
[[ℤInt(k) for k in 0:n] for n in 0:6]

7-element Array{Array{Nemo.fmpz,1},1}:
 [0]                  
 [0, 1]               
 [0, 1, 2]            
 [0, 1, 2, 3]         
 [0, 1, 2, 3, 4]      
 [0, 1, 2, 3, 4, 5]   
 [0, 1, 2, 3, 4, 5, 6]

In [4]:
ℤSeq(7, n -> ℤInt(n))

7-element Array{Nemo.fmpz,1}:
 0
 1
 2
 3
 4
 5
 6

In [5]:
T = Telescope(6, n -> ℤInt(n)) 
Println.(T)

[0]
[0, 1]
[0, 1, 2]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 5]


6-element Array{Nothing,1}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

The shape of the triangle is /not/ fixed. It includes the cases denoted by the OEIS keywords *tabl* and *tabf*. We allow the empty sequence to be element of an integer triangle.

In [6]:
function Divisors(n) 
    n == 0 && return ℤSeq()
    [ℤInt(d) for d in 1:n if rem(n, d) == 0]
end

Divisors (generic function with 1 method)

In [7]:
ℤTri(9, Divisors)

9-element Array{Array{Nemo.fmpz,1},1}:
 []          
 [1]         
 [1, 2]      
 [1, 3]      
 [1, 2, 4]   
 [1, 5]      
 [1, 2, 3, 6]
 [1, 7]      
 [1, 2, 4, 8]

In [8]:
D = ℤTri(9, Divisors)
isa(D, ℤTri)

true

In [9]:
import Base.sum
sum(T::ℤTri) = sum.(T)

sum (generic function with 15 methods)

In [10]:
sum(Divisors.(0:7))

8-element Array{Nemo.fmpz,1}:
 0 
 1 
 3 
 4 
 7 
 6 
 12
 8 

In [11]:
S = ℤSeq(5) 

5-element Array{Nemo.fmpz,1}:
 #undef
 #undef
 #undef
 #undef
 #undef

In [12]:
T = ℤTri(5) 

5-element Array{Array{Nemo.fmpz,1},1}:
 #undef
 #undef
 #undef
 #undef
 #undef

If the keyword 'reg' is set to true, the constructor returns an uninitialized regular triangle.

In [13]:
T = ℤTri(6, reg=true) 

6-element Array{Array{Nemo.fmpz,1},1}:
 [#undef]                                        
 [#undef, #undef]                                
 [#undef, #undef, #undef]                        
 [#undef, #undef, #undef, #undef]                
 [#undef, #undef, #undef, #undef, #undef]        
 [#undef, #undef, #undef, #undef, #undef, #undef]

In [14]:
for n in 1:6 T[n] = [ℤInt(k) for k in 1:n] end
T

6-element Array{Array{Nemo.fmpz,1},1}:
 [1]               
 [1, 2]            
 [1, 2, 3]         
 [1, 2, 3, 4]      
 [1, 2, 3, 4, 5]   
 [1, 2, 3, 4, 5, 6]

In [15]:
T[3]

3-element Array{Nemo.fmpz,1}:
 1
 2
 3

In [16]:
T[3][2]

2

In [17]:
isa(T, ℤTri) |> println
isa(T[3], ℤSeq) |> println
isa(T[3][2], ℤInt) |> println

true
true
true


In [18]:
function LahIndexed(n, k)
    function recLah(n, k)
        k  < 0 && return ℤInt(0)
        k == n && return ℤInt(1)
        recLah(n-1, k-1) + recLah(n-1, k)*(n+k-1)
    end
    recLah(n, k)
end

[[LahIndexed(n, k) for k in 0:n] for n in 0:6]

7-element Array{Array{Nemo.fmpz,1},1}:
 [1]                             
 [0, 1]                          
 [0, 2, 1]                       
 [0, 6, 6, 1]                    
 [0, 24, 36, 12, 1]              
 [0, 120, 240, 120, 20, 1]       
 [0, 720, 1800, 1200, 300, 30, 1]

Thus LahNumbers(n) returns an n-element Array{Array{ℤInt,1},1}
which is according to our definiton a triangle with *n* rows.

In [19]:
const CacheLah = Dict{Int, ℤSeq}([0 => [ℤInt(1)]])

function LahNumbers(n)
    haskey(CacheLah, n) && return CacheLah[n]
    prevrow = LahNumbers(n-1)
    row = ℤSeq(n+1)
    row[1] = 0; row[n+1] = 1
    for k in 2:n
        row[k] = prevrow[k-1] + prevrow[k]*(n+k-2)    
    end
    CacheLah[n] = row
end

LahNumbers (generic function with 1 method)

In [20]:
LahNumbers(5)

6-element Array{Nemo.fmpz,1}:
 0  
 120
 240
 120
 20 
 1  

In [21]:
println(CacheLah)

Dict(0 => [1],4 => [0, 24, 36, 12, 1],2 => [0, 2, 1],3 => [0, 6, 6, 1],5 => [0, 120, 240, 120, 20, 1],1 => [0, 1])


Let's check the time and space consumtion: 

In [22]:
@time LahNumbers(100);

  0.003886 seconds (10.18 k allocations: 212.766 KiB)


In [23]:
LahNumbers(n, k) = LahNumbers(n+1)[k+1]
LahNumbers(5, 3)

1200

In [24]:
methods(LahNumbers)

# 2 methods for generic function "LahNumbers":
[1] LahNumbers(n) in Main at In[19]:4
[2] LahNumbers(n, k) in Main at In[23]:1

In [25]:
function LahTriangle(size) 
    length(CacheLah) < size && LahNumbers(size)
    [CacheLah[n] for n in 0:size-1] 
end

LahTriangle (generic function with 1 method)

In [26]:
methods(LahTriangle)

# 1 method for generic function "LahTriangle":
[1] LahTriangle(size) in Main at In[25]:2

In [27]:
T = LahTriangle(7)
length(T)

7

In [28]:
evensum(A) = sum(A[1:2:end]) 
oddsum(A)  = sum(A[2:2:end])
altsum(A)  = evensum(A) - oddsum(A)
evensum(T::ℤTri) = evensum.(T)
oddsum(T::ℤTri)  = oddsum.(T)
altsum(T::ℤTri)  = evensum(T) - oddsum(T)

altsum (generic function with 2 methods)

In [29]:
T = LahTriangle(10)
Println.([sum(T), evensum(T), oddsum(T), altsum(T)]);

[1, 1, 3, 13, 73, 501, 4051, 37633, 394353, 4596553]
[1, 0, 1, 6, 37, 260, 2101, 19362, 201097, 2326536]
[0, 1, 2, 7, 36, 241, 1950, 18271, 193256, 2270017]
[1, -1, -1, -1, 1, 19, 151, 1091, 7841, 56519]


In [30]:
middle(A) = A[div(end + 1, 2)] 
middle(T::ℤTri) = middle.(T)

middle(LahTriangle(10)) |> Println

[1, 0, 2, 6, 36, 240, 1200, 12600, 58800, 846720]


In [31]:
central(T::ℤTri) = middle.(T[1:2:end])

central(LahTriangle(16)) |> Println

[1, 2, 36, 1200, 58800, 3810240, 307359360, 29682132480]


In [32]:
function DiagonalTriangle(T::ℤTri)
    dim = length(T)
    U = ℤTri(dim)
    for n in 1:dim
        R = ℤSeq(div(n+1,2))
        for k in 0:div(n-1,2)
            R[k+1] = T[n-k][k+1]
        end
        U[n] = R
    end
    U
end

DiagonalTriangle (generic function with 1 method)

In [33]:
T = LahTriangle(10) 
DiagonalTriangle(T)

10-element Array{Array{Nemo.fmpz,1},1}:
 [1]                        
 [0]                        
 [0, 1]                     
 [0, 2]                     
 [0, 6, 1]                  
 [0, 24, 6]                 
 [0, 120, 36, 1]            
 [0, 720, 240, 12]          
 [0, 5040, 1800, 120, 1]    
 [0, 40320, 15120, 1200, 20]

In [34]:
diagsum(T) = sum(diag(T))
diagsum(LahTriangle(9))

UndefVarError: UndefVarError: diag not defined

In [35]:
leftside(A)  = A[1] 
rightside(A) = A[end] 

rightside(T::ℤTri) = rightside.(T)
leftside( T::ℤTri) = leftside.(T)

leftside (generic function with 2 methods)

In [36]:
function profile(T::ℤTri)
    println("Triangle: ");
    for row in T Println(row) end; println()
    print("Sum:      "); sum(T)       |> Println 
    print("EvenSum:  "); evensum(T)   |> Println 
    print("OddSum:   "); oddsum(T)    |> Println 
    print("AltSum:   "); altsum(T)    |> Println 
    print("DiagSum:  "); diagsum(T)   |> Println 
    print("Middle:   "); middle(T)    |> Println 
    print("Central:  "); central(T)   |> Println 
    print("LeftSide: "); leftside(T)  |> Println 
    print("RightSide:"); rightside(T) |> Println 
end

profile (generic function with 1 method)

In [37]:
profile(LahTriangle(10))

Triangle: 
[1]
[0, 1]
[0, 2, 1]
[0, 6, 6, 1]
[0, 24, 36, 12, 1]
[0, 120, 240, 120, 20, 1]
[0, 720, 1800, 1200, 300, 30, 1]
[0, 5040, 15120, 12600, 4200, 630, 42, 1]
[0, 40320, 141120, 141120, 58800, 11760, 1176, 56, 1]
[0, 362880, 1451520, 1693440, 846720, 211680, 28224, 2016, 72, 1]

Sum:      [1, 1, 3, 13, 73, 501, 4051, 37633, 394353, 4596553]
EvenSum:  [1, 0, 1, 6, 37, 260, 2101, 19362, 201097, 2326536]
OddSum:   [0, 1, 2, 7, 36, 241, 1950, 18271, 193256, 2270017]
AltSum:   [1, -1, -1, -1, 1, 19, 151, 1091, 7841, 56519]
DiagSum:  

UndefVarError: UndefVarError: diag not defined

In [38]:
invT = InverseTriangle(T)      
isa(invT, ℤTri) |> println
invT

true


10-element Array{Array{Nemo.fmpz,1},1}:
 [1]                                                                  
 [0, 1]                                                               
 [0, -2, 1]                                                           
 [0, 6, -6, 1]                                                        
 [0, -24, 36, -12, 1]                                                 
 [0, 120, -240, 120, -20, 1]                                          
 [0, -720, 1800, -1200, 300, -30, 1]                                  
 [0, 5040, -15120, 12600, -4200, 630, -42, 1]                         
 [0, -40320, 141120, -141120, 58800, -11760, 1176, -56, 1]            
 [0, 362880, -1451520, 1693440, -846720, 211680, -28224, 2016, -72, 1]

In [39]:
Apery = [
            ℤInt(1),
            5,
            73,
            1445,
            33001,
            819005,
            21460825,
            584307365
        ]

BinomialTransform(Apery) |> Println

[1, 6, 84, 1680, 39240, 999216, 26899896, 752939424]


In [40]:
dim = 8
TraitCard(NarayanaTriangle, NarayanaTransform, dim)


NarayanaTriangle 

Triangle:  [1, 0, 1, 0, 1, 1, 0, 1]
Sum:       [1, 1, 2, 5, 14, 42, 132, 429]
EvenSum:   [1, 0, 1, 3, 7, 20, 66, 217]
OddSum:    [0, 1, 1, 2, 7, 22, 66, 212]
AltSum:    [1, -1, 0, 1, 0, -2, 0, 5]
DiagSum:   [1, 0, 1, 1, 2, 4, 8, 17]
Central:   [1, 1, 6, 50, 490]
LeftSide:  [1, 0, 0, 0, 0, 0, 0, 0]
RightSide: [1, 1, 1, 1, 1, 1, 1, 1]
PosHalf:   [1, 1, 3, 11, 45, 197, 903, 4279]
NegHalf:   [1, 1, -1, -1, 5, -3, -21, 51]
N0TS:      [0, 1, 3, 10, 35, 126, 462, 1716]
NATS:      [1, 2, 5, 15, 49, 168, 594, 2145]


Triangle:

[1]
[0, 1]
[0, 1, 1]
[0, 1, 3, 1]
[0, 1, 6, 6, 1]
[0, 1, 10, 20, 10, 1]
[0, 1, 15, 50, 50, 15, 1]
[0, 1, 21, 105, 175, 105, 21, 1]

Flat triangle:  [1, 0, 1, 0, 1, 1, 0, 1]

Inverse triangle:

[1]
[0, 1]
[0, -1, 1]
[0, 2, -3, 1]
[0, -7, 12, -6, 1]
[0, 39, -70, 40, -10, 1]
[0, -321, 585, -350, 100, -15, 1]
[0, 3681, -6741, 4095, -1225, 210, -21, 1]

Inverse:  [1, 0, 1, 0, -1, 1, 0, 2]

Diagonal triangle:

[1]
[0]
[0, 1]
[0, 1]
[0, 1, 1]
[0, 1, 3]
[0, 1

In [41]:
TraitCard(LaguerreTriangle, LaguerreTransform, dim)


LaguerreTriangle 

Triangle:  [1, 1, 1, 2, 4, 1, 6, 18]
Sum:       [1, 2, 7, 34, 209, 1546, 13327, 130922]
EvenSum:   [1, 1, 3, 15, 97, 745, 6571, 65359]
OddSum:    [0, 1, 4, 19, 112, 801, 6756, 65563]
AltSum:    [1, 0, -1, -4, -15, -56, -185, -204]
DiagSum:   [1, 1, 3, 10, 43, 225, 1393, 9976]
Central:   [1, 4, 72, 2400]
LeftSide:  [1, 1, 2, 6, 24, 120, 720, 5040]
RightSide: [1, 1, 1, 1, 1, 1, 1, 1]
PosHalf:   [1, 3, 17, 139, 1473, 19091, 291793, 5129307]
NegHalf:   [1, -1, 1, 7, -127, 1711, -23231, 334391]
N0TS:      [0, 1, 6, 39, 292, 2505, 24306, 263431]
NATS:      [1, 3, 13, 73, 501, 4051, 37633, 394353]


Triangle:

[1]
[1, 1]
[2, 4, 1]
[6, 18, 9, 1]
[24, 96, 72, 16, 1]
[120, 600, 600, 200, 25, 1]
[720, 4320, 5400, 2400, 450, 36, 1]
[5040, 35280, 52920, 29400, 7350, 882, 49, 1]

Flat triangle:  [1, 1, 1, 2, 4, 1, 6, 18]

Inverse triangle:

[1]
[-1, 1]
[2, -4, 1]
[-6, 18, -9, 1]
[24, -96, 72, -16, 1]
[-120, 600, -600, 200, -25, 1]
[720, -4320, 5400, -2400, 450, -36, 1]
[-5040, 35

In [42]:
TraitCard(MotzkinTriangle, MotzkinTransform, dim)


MotzkinTriangle 

Triangle:  [1, 1, 1, 2, 2, 1, 4, 5]
Sum:       [1, 2, 5, 13, 35, 96, 267, 750]
EvenSum:   [1, 1, 3, 7, 19, 51, 141, 393]
OddSum:    [0, 1, 2, 6, 16, 45, 126, 357]
AltSum:    [1, 0, 1, 1, 3, 6, 15, 36]
DiagSum:   [1, 1, 3, 6, 15, 36, 91, 232]
Central:   [1, 2, 9, 44]
LeftSide:  [1, 1, 2, 4, 9, 21, 51, 127]
RightSide: [1, 1, 1, 1, 1, 1, 1, 1]
PosHalf:   [1, 3, 13, 59, 285, 1419, 7245, 37659]
NegHalf:   [1, -1, 5, -17, 77, -345, 1653, -8097]
N0TS:      [0, 1, 4, 14, 46, 147, 462, 1437]
NATS:      [1, 3, 9, 27, 81, 243, 729, 2187]


Triangle:

[1]
[1, 1]
[2, 2, 1]
[4, 5, 3, 1]
[9, 12, 9, 4, 1]
[21, 30, 25, 14, 5, 1]
[51, 76, 69, 44, 20, 6, 1]
[127, 196, 189, 133, 70, 27, 7, 1]

Flat triangle:  [1, 1, 1, 2, 2, 1, 4, 5]

Inverse triangle:

[1]
[-1, 1]
[0, -2, 1]
[1, 1, -3, 1]
[-1, 2, 3, -4, 1]
[0, -4, 2, 6, -5, 1]
[1, 2, -9, 0, 10, -6, 1]
[-1, 3, 9, -15, -5, 15, -7, 1]

Inverse:  [1, -1, 1, 0, -2, 1, 1, 1]

Diagonal triangle:

[1]
[1]
[2, 1]
[4, 2]
[9, 5, 1]
[21, 12, 3]
[5

In [43]:
TraitCard(JacobsthalTriangle, JacobsthalTransform, dim)


JacobsthalTriangle 

Triangle:  [1, 1, 1, 1, 2, 1, 3, 5]
Sum:       [1, 2, 4, 12, 32, 88, 240, 656]
EvenSum:   [1, 1, 2, 6, 16, 44, 120, 328]
OddSum:    [0, 1, 2, 6, 16, 44, 120, 328]
AltSum:    [1, 0, 0, 0, 0, 0, 0, 0]
DiagSum:   [1, 1, 2, 5, 11, 26, 59, 137]
Central:   [1, 2, 10, 52]
LeftSide:  [1, 1, 1, 3, 5, 11, 21, 43]
RightSide: [1, 1, 1, 1, 1, 1, 1, 1]
PosHalf:   [1, 3, 9, 51, 225, 1083, 5049, 23811]
NegHalf:   [1, -1, 1, -9, 17, -89, 225, -937]
N0TS:      [0, 1, 4, 14, 48, 156, 496, 1544]
NATS:      [1, 3, 8, 26, 80, 244, 736, 2200]


Triangle:

[1]
[1, 1]
[1, 2, 1]
[3, 5, 3, 1]
[5, 12, 10, 4, 1]
[11, 27, 28, 16, 5, 1]
[21, 62, 75, 52, 23, 6, 1]
[43, 137, 193, 159, 85, 31, 7, 1]

Flat triangle:  [1, 1, 1, 1, 2, 1, 3, 5]

Inverse triangle:

[1]
[-1, 1]
[1, -2, 1]
[-1, 1, -3, 1]
[1, 4, 2, -4, 1]
[-1, -7, 10, 4, -5, 1]
[1, -14, -25, 16, 7, -6, 1]
[-1, 65, -21, -55, 21, 11, -7, 1]

Inverse:  [1, -1, 1, 1, -2, 1, -1, 1]

Diagonal triangle:

[1]
[1]
[1, 1]
[3, 2]
[5, 5, 1]
[11, 12, 